In [51]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 25: Sea Cucumber ---</h2><p>This is it: the bottom of the ocean trench, the last place the sleigh keys could be. Your submarine's experimental antenna <em>still isn't boosted enough</em> to detect the keys, but they <em>must</em> be here. All you need to do is <em>reach the seafloor</em> and find them.</p>
<p>At least, you'd touch down on the seafloor if you could; unfortunately, it's completely covered by two large herds of <a href="https://en.wikipedia.org/wiki/Sea_cucumber" target="_blank">sea cucumbers</a>, and there isn't an open space large enough for your submarine.</p>
<p>You suspect that the Elves must have done this before, because just then you discover the phone number of a deep-sea marine biologist on a handwritten note taped to the wall of the submarine's cockpit.</p>
<p>"Sea cucumbers? Yeah, they're probably hunting for food. But don't worry, they're predictable critters: they move in perfectly straight lines, only moving forward when there's space to do so. They're actually quite polite!"</p>
<p>You explain that you'd like to predict when you could land your submarine.</p>
<p>"Oh that's easy, they'll eventually pile up and leave enough space for-- wait, did you say submarine? And the only place with that many sea cucumbers would be at the very bottom of the Mariana--" You hang up the phone.</p>
<p>There are two herds of sea cucumbers sharing the same region; one always moves <em>east</em> (<code>&gt;</code>), while the other always moves <em>south</em> (<code>v</code>). Each location can contain at most one sea cucumber; the remaining locations are <em>empty</em> (<code>.</code>). The submarine helpfully generates a map of the situation (your puzzle input). For example:</p>
<pre><code>v...&gt;&gt;.vv&gt;
.vv&gt;&gt;.vv..
&gt;&gt;.&gt;v&gt;...v
&gt;&gt;v&gt;&gt;.&gt;.v.
v&gt;v.vv.v..
&gt;.&gt;&gt;..v...
.vv..&gt;.&gt;v.
v.v..&gt;&gt;v.v
....v..v.&gt;
</code></pre>
<p>Every <em>step</em>, the sea cucumbers in the east-facing herd attempt to move forward one location, then the sea cucumbers in the south-facing herd attempt to move forward one location. When a herd moves forward, every sea cucumber in the herd first simultaneously considers whether there is a sea cucumber in the adjacent location it's facing (even another sea cucumber facing the same direction), and then every sea cucumber facing an empty location simultaneously moves into that location.</p>
<p>So, in a situation like this:</p>
<pre><code>...&gt;&gt;&gt;&gt;&gt;...</code></pre>
<p>After one step, only the rightmost sea cucumber would have moved:</p>
<pre><code>...&gt;&gt;&gt;&gt;.&gt;..</code></pre>
<p>After the next step, two sea cucumbers move:</p>
<pre><code>...&gt;&gt;&gt;.&gt;.&gt;.</code></pre>
<p>During a single step, the east-facing herd moves first, then the south-facing herd moves. So, given this situation:</p>
<pre><code>..........
.&gt;v....v..
.......&gt;..
..........
</code></pre>
<p>After a single step, of the sea cucumbers on the left, only the south-facing sea cucumber has moved (as it wasn't out of the way in time for the east-facing cucumber on the left to move), but both sea cucumbers on the right have moved (as the east-facing sea cucumber moved out of the way of the south-facing sea cucumber):</p>
<pre><code>..........
.&gt;........
..v....v&gt;.
..........
</code></pre>
<p>Due to <em>strong water currents</em> in the area, sea cucumbers that move off the right edge of the map appear on the left edge, and sea cucumbers that move off the bottom edge of the map appear on the top edge. Sea cucumbers always check whether their destination location is empty before moving, even if that destination is on the opposite side of the map:</p>
<pre><code>Initial state:
...&gt;...
.......
......&gt;
v.....&gt;
......&gt;
.......
..vvv..

After 1 step:
..vv&gt;..
.......
&gt;......
v.....&gt;
&gt;......
.......
....v..

After 2 steps:
....v&gt;.
..vv...
.&gt;.....
......&gt;
v&gt;.....
.......
.......

After 3 steps:
......&gt;
..v.v..
..&gt;v...
&gt;......
..&gt;....
v......
.......

After 4 steps:
&gt;......
..v....
..&gt;.v..
.&gt;.v...
...&gt;...
.......
v......
</code></pre>

<p>To find a safe place to land your submarine, the sea cucumbers need to stop moving. Again consider the first example:</p>
<pre><code>Initial state:
v...&gt;&gt;.vv&gt;
.vv&gt;&gt;.vv..
&gt;&gt;.&gt;v&gt;...v
&gt;&gt;v&gt;&gt;.&gt;.v.
v&gt;v.vv.v..
&gt;.&gt;&gt;..v...
.vv..&gt;.&gt;v.
v.v..&gt;&gt;v.v
....v..v.&gt;

After 1 step:
....&gt;.&gt;v.&gt;
v.v&gt;.&gt;v.v.
&gt;v&gt;&gt;..&gt;v..
&gt;&gt;v&gt;v&gt;.&gt;.v
.&gt;v.v...v.
v&gt;&gt;.&gt;vvv..
..v...&gt;&gt;..
vv...&gt;&gt;vv.
&gt;.v.v..v.v

After 2 steps:
&gt;.v.v&gt;&gt;..v
v.v.&gt;&gt;vv..
&gt;v&gt;.&gt;.&gt;.v.
&gt;&gt;v&gt;v.&gt;v&gt;.
.&gt;..v....v
.&gt;v&gt;&gt;.v.v.
v....v&gt;v&gt;.
.vv..&gt;&gt;v..
v&gt;.....vv.

After 3 steps:
v&gt;v.v&gt;.&gt;v.
v...&gt;&gt;.v.v
&gt;vv&gt;.&gt;v&gt;..
&gt;&gt;v&gt;v.&gt;.v&gt;
..&gt;....v..
.&gt;.&gt;v&gt;v..v
..v..v&gt;vv&gt;
v.v..&gt;&gt;v..
.v&gt;....v..

After 4 steps:
v&gt;..v.&gt;&gt;..
v.v.&gt;.&gt;.v.
&gt;vv.&gt;&gt;.v&gt;v
&gt;&gt;.&gt;..v&gt;.&gt;
..v&gt;v...v.
..&gt;&gt;.&gt;vv..
&gt;.v.vv&gt;v.v
.....&gt;&gt;vv.
vvv&gt;...v..

After 5 steps:
vv&gt;...&gt;v&gt;.
v.v.v&gt;.&gt;v.
&gt;.v.&gt;.&gt;.&gt;v
&gt;v&gt;.&gt;..v&gt;&gt;
..v&gt;v.v...
..&gt;.&gt;&gt;vvv.
.&gt;...v&gt;v..
..v.v&gt;&gt;v.v
v.v.&gt;...v.

...

After 10 steps:
..&gt;..&gt;&gt;vv.
v.....&gt;&gt;.v
..v.v&gt;&gt;&gt;v&gt;
v&gt;.&gt;v.&gt;&gt;&gt;.
..v&gt;v.vv.v
.v.&gt;&gt;&gt;.v..
v.v..&gt;v&gt;..
..v...&gt;v.&gt;
.vv..v&gt;vv.

...

After 20 steps:
v&gt;.....&gt;&gt;.
&gt;vv&gt;.....v
.&gt;v&gt;v.vv&gt;&gt;
v&gt;&gt;&gt;v.&gt;v.&gt;
....vv&gt;v..
.v.&gt;&gt;&gt;vvv.
..v..&gt;&gt;vv.
v.v...&gt;&gt;.v
..v.....v&gt;

...

After 30 steps:
.vv.v..&gt;&gt;&gt;
v&gt;...v...&gt;
&gt;.v&gt;.&gt;vv.&gt;
&gt;v&gt;.&gt;.&gt;v.&gt;
.&gt;..v.vv..
..v&gt;..&gt;&gt;v.
....v&gt;..&gt;v
v.v...&gt;vv&gt;
v.v...&gt;vvv

...

After 40 steps:
&gt;&gt;v&gt;v..v..
..&gt;&gt;v..vv.
..&gt;&gt;&gt;v.&gt;.v
..&gt;&gt;&gt;&gt;vvv&gt;
v.....&gt;...
v.v...&gt;v&gt;&gt;
&gt;vv.....v&gt;
.&gt;v...v.&gt;v
vvv.v..v.&gt;

...

After 50 steps:
..&gt;&gt;v&gt;vv.v
..v.&gt;&gt;vv..
v.&gt;&gt;v&gt;&gt;v..
..&gt;&gt;&gt;&gt;&gt;vv.
vvv....&gt;vv
..v....&gt;&gt;&gt;
v&gt;.......&gt;
.vv&gt;....v&gt;
.&gt;v.vv.v..

...

After 55 steps:
..&gt;&gt;v&gt;vv..
..v.&gt;&gt;vv..
..&gt;&gt;v&gt;&gt;vv.
..&gt;&gt;&gt;&gt;&gt;vv.
v......&gt;vv
v&gt;v....&gt;&gt;v
vvv...&gt;..&gt;
&gt;vv.....&gt;.
.&gt;v.vv.v..

After 56 steps:
..&gt;&gt;v&gt;vv..
..v.&gt;&gt;vv..
..&gt;&gt;v&gt;&gt;vv.
..&gt;&gt;&gt;&gt;&gt;vv.
v......&gt;vv
v&gt;v....&gt;&gt;v
vvv....&gt;.&gt;
&gt;vv......&gt;
.&gt;v.vv.v..

After 57 steps:
..&gt;&gt;v&gt;vv..
..v.&gt;&gt;vv..
..&gt;&gt;v&gt;&gt;vv.
..&gt;&gt;&gt;&gt;&gt;vv.
v......&gt;vv
v&gt;v....&gt;&gt;v
vvv.....&gt;&gt;
&gt;vv......&gt;
.&gt;v.vv.v..

After 58 steps:
..&gt;&gt;v&gt;vv..
..v.&gt;&gt;vv..
..&gt;&gt;v&gt;&gt;vv.
..&gt;&gt;&gt;&gt;&gt;vv.
v......&gt;vv
v&gt;v....&gt;&gt;v
vvv.....&gt;&gt;
&gt;vv......&gt;
.&gt;v.vv.v..
</code></pre>

<p>In this example, the sea cucumbers stop moving after <code><em>58</em></code> steps.</p>
<p>Find somewhere safe to land your submarine. <em>What is the first step on which no sea cucumbers move?</em></p>
</article>


In [52]:
from copy import deepcopy

type Location = tuple[int, int]


class Region:
    def __init__(self, s: str) -> None:
        self.grid = self.parse(s)
        self.rows, self.cols = len(self.grid), len(self.grid[0])
        self.east, self.south = self.cucumbers(self.grid)

    def first_step_no_cucumbers_move(self) -> int:
        step = 1
        while self.step():
            step += 1
        return step

    def step(self) -> bool:
        east_changed = self.move_east()
        south_changed = self.move_south()
        return east_changed or south_changed

    def move_east(self) -> bool:
        east = set()
        changed = False
        for r, c in self.east:
            loc = r, (c + 1) % self.cols
            if loc not in self.east and loc not in self.south:
                east.add(loc)
                _, c1 = loc
                self.grid[r][c], self.grid[r][c1] = (
                    self.grid[r][c1],
                    self.grid[r][c],
                )
                changed = True
            else:
                east.add((r, c))

        self.east = east
        return changed

    def move_south(self) -> bool:
        south = set()
        changed = False

        for r, c in self.south:
            loc = (r + 1) % self.rows, c
            if loc not in self.east and loc not in self.south:
                south.add(loc)
                r1, _ = loc
                self.grid[r][c], self.grid[r1][c] = (
                    self.grid[r1][c],
                    self.grid[r][c],
                )
                changed = True
            else:
                south.add((r, c))

        self.south = south
        return changed

    def __eq__(self, other: Region) -> bool:
        return tuple(self.__dict__.values()) == tuple(other.__dict__.values())

    def __str__(self) -> str:
        return f'\n{"\n".join("".join(l) for l in self.grid)}\n\n'

    @staticmethod
    def cucumbers(grid: list[list[str]]) -> tuple[set[Location], set[Location]]:
        east, south = set(), set()

        for r, row in enumerate(grid):
            for c, value in enumerate(row):
                if value == ">":
                    east.add((r, c))
                elif value == "v":
                    south.add((r, c))

        return east, south

    @staticmethod
    def parse(s: str) -> list[list[str]]:
        return [list(l.strip()) for l in s.strip().splitlines()]


example = """
v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>
"""

assert Region(example).first_step_no_cucumbers_move() == 58

In [53]:
region_step_tests = [
    {
        "name": "Simple example 1 a",
        "s": """...>>>>>...""",
        "expected": Region("...>>>>.>.."),
    },
    {
        "name": "Simple example 1 b",
        "s": """...>>>>.>..""",
        "expected": Region("...>>>.>.>."),
    },
    {
        "name": "Simple example 2",
        "s": """
            ..........
            .>v....v..
            .......>..
            ..........
        """,
        "expected": Region(
            """
                ..........
                .>........
                ..v....v>.
                ..........
            """
        ),
    },
    {
        "name": "Simple 3 example after 1 step",
        "s": """
            ...>...
            .......
            ......>
            v.....>
            ......>
            .......
            ..vvv..
        """,
        "expected": Region(
            """
                ..vv>..
                .......
                >......
                v.....>
                >......
                .......
                ....v..
            """
        ),
    },
]

PRT = False


def assert_funct(actual: Region, expected: Region) -> bool:
    if PRT:
        print(f"actual  : {actual.grid}")
        print(f"expected: {expected.grid}")
        print(f"actual  : {actual.rows}")
        print(f"expected: {expected.rows}")
        print(f"actual  : {actual.cols}")
        print(f"expected: {expected.cols}")
        print(f"actual  : {actual.east}")
        print(f"expected: {expected.east}")
        print(f"actual  : {actual.south}")
        print(f"expected: {expected.south}")
    return actual == expected


@test(tests=region_step_tests, assert_funct=assert_funct)
def region_step_test(s: str) -> Region:
    region = Region(s)
    region.step()
    return region


Test Simple example 1 a passed, for region_step_test.
Test Simple example 1 b passed, for region_step_test.
Test Simple example 2 passed, for region_step_test.
Test Simple 3 example after 1 step passed, for region_step_test.
Success


In [54]:
region_constructor_tests = [
    {
        "name": "Simple example 1",
        "s": """...>>>>>...""",
        "expected": (
            [
                [".", ".", ".", ">", ">", ">", ">", ">", ".", ".", "."],
            ],
            {(0, 3), (0, 4), (0, 5), (0, 6), (0, 7)},
            set(),
        ),
    },
    {
        "name": "Simple example 2",
        "s": """
            ..........
            .>v....v..
            .......>..
            ..........
        """,
        "expected": (
            [
                [".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
                [".", ">", "v", ".", ".", ".", ".", "v", ".", "."],
                [".", ".", ".", ".", ".", ".", ".", ">", ".", "."],
                [".", ".", ".", ".", ".", ".", ".", ".", ".", "."],
            ],
            {(1, 1), (2, 7)},
            {(1, 2), (1, 7)},
        ),
    },
    {
        "name": "Main example",
        "s": """
            v...>>.vv>
            .vv>>.vv..
            >>.>v>...v
            >>v>>.>.v.
            v>v.vv.v..
            >.>>..v...
            .vv..>.>v.
            v.v..>>v.v
            ....v..v.>
        """,
        "expected": (
            [
                ["v", ".", ".", ".", ">", ">", ".", "v", "v", ">"],
                [".", "v", "v", ">", ">", ".", "v", "v", ".", "."],
                [">", ">", ".", ">", "v", ">", ".", ".", ".", "v"],
                [">", ">", "v", ">", ">", ".", ">", ".", "v", "."],
                ["v", ">", "v", ".", "v", "v", ".", "v", ".", "."],
                [">", ".", ">", ">", ".", ".", "v", ".", ".", "."],
                [".", "v", "v", ".", ".", ">", ".", ">", "v", "."],
                ["v", ".", "v", ".", ".", ">", ">", "v", ".", "v"],
                [".", ".", ".", ".", "v", ".", ".", "v", ".", ">"],
            ],
            {
                (0, 4),
                (0, 5),
                (0, 9),
                (1, 3),
                (1, 4),
                (2, 0),
                (2, 1),
                (2, 3),
                (2, 5),
                (3, 0),
                (3, 1),
                (3, 3),
                (3, 4),
                (3, 6),
                (4, 1),
                (5, 0),
                (5, 2),
                (5, 3),
                (6, 5),
                (6, 7),
                (7, 5),
                (7, 6),
                (8, 9),
            },
            {
                (0, 0),
                (0, 7),
                (0, 8),
                (1, 1),
                (1, 2),
                (1, 6),
                (1, 7),
                (2, 4),
                (2, 9),
                (3, 2),
                (3, 8),
                (4, 0),
                (4, 2),
                (4, 4),
                (4, 5),
                (4, 7),
                (5, 6),
                (6, 1),
                (6, 2),
                (6, 8),
                (7, 0),
                (7, 2),
                (7, 7),
                (7, 9),
                (8, 4),
                (8, 7),
            },
        ),
    },
]


@test(tests=region_constructor_tests)
def region_constructor_test(s: str) -> dict[str, int]:
    region = Region(s)
    return region.grid, region.east, region.south


Test Simple example 1 passed, for region_constructor_test.
Test Simple example 2 passed, for region_constructor_test.
Test Main example passed, for region_constructor_test.
Success


In [55]:
with open("../input/day25.txt") as f:
    puzzle = f.read()

print(f"Step1: {Region(puzzle).first_step_no_cucumbers_move()}")

Step1: 520


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>520</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Suddenly, the experimental antenna control console lights up:</p>
<pre><code><em>Sleigh keys detected!</em></code></pre>
<p>According to the console, the keys are <em>directly under the submarine</em>. <span title="Thanks to the deep-sea marine biologist, who apparently works at the Biham-Middleton-Levine oceanic research institute.">You landed</span> right on them! Using a robotic arm on the submarine, you move the sleigh keys into the airlock.</p>
<p>Now, you just need to get them to Santa in time to save Christmas! You check your clock - it <em>is</em> Christmas. There's no way you can get them back to the surface in time.</p>
<p>Just as you start to lose hope, you notice a button on the sleigh keys: <em>remote start</em>. You can start the sleigh from the bottom of the ocean! You just need some way to <em>boost the signal</em> from the keys so it actually reaches the sleigh. Good thing the submarine has that experimental antenna! You'll definitely need <em class="star">50 stars</em> to boost it that far, though.</p>
<p>The experimental antenna control console lights up again:</p>
<pre><code><em>Energy source detected.
Integrating energy source from device "sleigh keys"...done.
Installing device drivers...done.
Recalibrating experimental antenna...done.
Boost strength due to matching signal phase: <em class="star">1 star</em>
</em></code></pre>
<p>Only <em class="star">49 stars</em> to go.</p>
</article>


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<body data-new-gr-c-s-check-loaded="14.1259.0" data-gr-ext-installed="">
<header><div><h1 class="title-global"><a href="/">Advent of Code</a></h1><nav><ul><li><a href="/2021/about">[About]</a></li><li><a href="/2021/events">[Events]</a></li><li><a href="https://cottonbureau.com/people/advent-of-code" target="_blank">[Shop]</a></li><li><a href="/2021/settings">[Settings]</a></li><li><a href="/2021/auth/logout">[Log Out]</a></li></ul></nav><div class="user">JanBergsma <span class="star-count">50*</span></div></div><div><h1 class="title-event">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<span class="title-event-wrap"></span><a href="/2021">2021</a><span class="title-event-wrap"></span></h1><nav><ul><li><a href="/2021">[Calendar]</a></li><li><a href="/2021/support">[AoC++]</a></li><li><a href="/2021/sponsors">[Sponsors]</a></li><li><a href="/2021/leaderboard">[Leaderboards]</a></li><li><a href="/2021/stats">[Stats]</a></li></ul></nav></div></header>

<div id="sidebar">
<div id="sponsor"><div class="quiet">Our <a href="/2021/sponsors">sponsors</a> help make Advent of Code possible:</div><div class="sponsor"><a href="/2021/sponsors/redirect?url=https%3A%2F%2Fjb%2Egg%2FAoC2021tips" target="_blank" onclick="if(ga)ga('send','event','sponsor','sidebar',this.href);" rel="noopener">JetBrains</a> - Get ready to jingle with Advent of Code in Kotlin! Have fun, learn new things, and win prizes. Believe in magic with Kotlin. Happy holidays! https://jb.gg/AoC</div></div>
</div><!--/sidebar-->

<main>
<article>
<p>You use all <span class="day-success">fifty stars</span> to boost the signal and remotely start the sleigh! Now, you just have to find your way back to the surface...</p><p>...did you know crab submarines come with colored lights?</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2021!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="https://was.tl/" target="_blank">my website</a> or through <a href="https://bsky.app/profile/was.tl" target="_blank">Bluesky</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2021/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://bsky.app/intent/compose?text=I+just+completed+all+25+days+of+Advent+of+Code+2021%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F" target="_blank">Bluesky</a>
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2021%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2021%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends or <a href="/2021">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf0" style="top:-1.55em; left:33.37vw; animation-delay:-13.89s;"></div><div class="sf sf2" style="top:-3.69em; left:6.93vw; animation-delay:-18.27s;"></div><div class="sf sf2" style="top:-3.66em; left:69.90vw; animation-delay:-13.34s;"></div><div class="sf sf1" style="top:-2.72em; left:21.37vw; animation-delay:-11.38s;"></div><div class="sf sf3" style="top:-4.92em; left:78.43vw; animation-delay:-11.75s;"></div><div class="sf sf2" style="top:-3.94em; left:75.58vw; animation-delay:-10.42s;"></div><div class="sf sf1" style="top:-2.80em; left:47.38vw; animation-delay:-8.62s;"></div><div class="sf sf2" style="top:-3.69em; left:68.25vw; animation-delay:-17.69s;"></div><div class="sf sf0" style="top:-1.02em; left:24.43vw; animation-delay:-8.34s;"></div><div class="sf sf1" style="top:-2.65em; left:9.47vw; animation-delay:-0.90s;"></div><div class="sf sf2" style="top:-3.53em; left:42.45vw; animation-delay:-12.40s;"></div><div class="sf sf2" style="top:-3.04em; left:67.94vw; animation-delay:-10.24s;"></div><div class="sf sf2" style="top:-3.32em; left:92.77vw; animation-delay:-14.93s;"></div><div class="sf sf3" style="top:-4.05em; left:31.18vw; animation-delay:-11.27s;"></div><div class="sf sf3" style="top:-4.71em; left:79.53vw; animation-delay:-1.52s;"></div><div class="sf sf1" style="top:-2.79em; left:17.68vw; animation-delay:-13.15s;"></div><div class="sf sf0" style="top:-1.59em; left:12.07vw; animation-delay:-0.68s;"></div><div class="sf sf3" style="top:-4.41em; left:62.95vw; animation-delay:-3.77s;"></div><div class="sf sf1" style="top:-2.63em; left:33.42vw; animation-delay:-16.80s;"></div><div class="sf sf3" style="top:-4.56em; left:73.78vw; animation-delay:-15.96s;"></div><div class="sf sf3" style="top:-4.60em; left:45.26vw; animation-delay:-4.43s;"></div><div class="sf sf3" style="top:-4.02em; left:22.15vw; animation-delay:-2.58s;"></div><div class="sf sf2" style="top:-3.24em; left:55.78vw; animation-delay:-4.31s;"></div><div class="sf sf3" style="top:-4.70em; left:67.57vw; animation-delay:-11.56s;"></div><div class="sf sf2" style="top:-3.85em; left:4.00vw; animation-delay:-13.83s;"></div><div class="sf sf3" style="top:-4.27em; left:33.58vw; animation-delay:-7.33s;"></div><div class="sf sf2" style="top:-3.42em; left:64.49vw; animation-delay:-6.46s;"></div><div class="sf sf3" style="top:-4.67em; left:75.88vw; animation-delay:-19.56s;"></div><div class="sf sf2" style="top:-3.52em; left:93.68vw; animation-delay:-8.15s;"></div><div class="sf sf2" style="top:-3.37em; left:61.37vw; animation-delay:-16.34s;"></div><div class="sf sf1" style="top:-2.61em; left:13.66vw; animation-delay:-1.87s;"></div><div class="sf sf1" style="top:-2.67em; left:88.73vw; animation-delay:-15.17s;"></div><div class="sf sf0" style="top:-1.89em; left:46.41vw; animation-delay:-6.11s;"></div><div class="sf sf2" style="top:-3.85em; left:7.68vw; animation-delay:-2.29s;"></div><div class="sf sf3" style="top:-4.30em; left:25.96vw; animation-delay:-17.71s;"></div><div class="sf sf0" style="top:-1.53em; left:70.28vw; animation-delay:-8.51s;"></div><div class="sf sf2" style="top:-3.07em; left:52.61vw; animation-delay:-5.02s;"></div><div class="sf sf0" style="top:-1.99em; left:18.47vw; animation-delay:-7.84s;"></div><div class="sf sf0" style="top:-1.19em; left:62.31vw; animation-delay:-3.52s;"></div><div class="sf sf3" style="top:-4.27em; left:23.19vw; animation-delay:-4.63s;"></div><div class="sf sf2" style="top:-3.83em; left:91.17vw; animation-delay:-2.65s;"></div><div class="sf sf2" style="top:-3.63em; left:1.01vw; animation-delay:-13.73s;"></div><div class="sf sf2" style="top:-3.98em; left:63.23vw; animation-delay:-0.16s;"></div><div class="sf sf3" style="top:-4.89em; left:69.92vw; animation-delay:-19.17s;"></div><div class="sf sf3" style="top:-4.79em; left:76.40vw; animation-delay:-18.02s;"></div><div class="sf sf2" style="top:-3.15em; left:41.01vw; animation-delay:-17.03s;"></div><div class="sf sf1" style="top:-2.24em; left:94.47vw; animation-delay:-2.82s;"></div><div class="sf sf2" style="top:-3.23em; left:18.68vw; animation-delay:-9.83s;"></div><div class="sf sf2" style="top:-3.22em; left:34.03vw; animation-delay:-12.56s;"></div><div class="sf sf3" style="top:-4.50em; left:11.32vw; animation-delay:-2.08s;"></div><div class="sf sf0" style="top:-1.72em; left:53.58vw; animation-delay:-1.88s;"></div><div class="sf sf3" style="top:-4.07em; left:90.10vw; animation-delay:-8.65s;"></div><div class="sf sf2" style="top:-3.34em; left:17.09vw; animation-delay:-17.92s;"></div><div class="sf sf3" style="top:-4.63em; left:12.80vw; animation-delay:-11.70s;"></div><div class="sf sf3" style="top:-4.96em; left:37.62vw; animation-delay:-10.50s;"></div><div class="sf sf1" style="top:-2.66em; left:39.56vw; animation-delay:-5.24s;"></div><div class="sf sf3" style="top:-4.79em; left:66.48vw; animation-delay:-1.73s;"></div><div class="sf sf0" style="top:-1.67em; left:81.69vw; animation-delay:-11.20s;"></div><div class="sf sf2" style="top:-3.23em; left:26.08vw; animation-delay:-12.67s;"></div><div class="sf sf1" style="top:-2.48em; left:14.17vw; animation-delay:-16.05s;"></div><div class="sf sf1" style="top:-2.63em; left:69.48vw; animation-delay:-7.69s;"></div><div class="sf sf3" style="top:-4.62em; left:91.63vw; animation-delay:-2.82s;"></div><div class="sf sf0" style="top:-1.17em; left:26.85vw; animation-delay:-10.65s;"></div><div class="sf sf3" style="top:-4.96em; left:46.51vw; animation-delay:-19.65s;"></div><div class="sf sf3" style="top:-4.13em; left:58.56vw; animation-delay:-11.34s;"></div><div class="sf sf3" style="top:-4.59em; left:42.11vw; animation-delay:-13.26s;"></div><div class="sf sf1" style="top:-2.43em; left:51.76vw; animation-delay:-15.59s;"></div><div class="sf sf1" style="top:-2.19em; left:51.75vw; animation-delay:-3.27s;"></div><div class="sf sf3" style="top:-4.11em; left:91.17vw; animation-delay:-17.68s;"></div><div class="sf sf0" style="top:-1.71em; left:94.08vw; animation-delay:-1.63s;"></div><div class="sf sf3" style="top:-4.41em; left:71.89vw; animation-delay:-10.09s;"></div><div class="sf sf0" style="top:-1.17em; left:32.74vw; animation-delay:-10.45s;"></div><div class="sf sf3" style="top:-4.70em; left:15.41vw; animation-delay:-12.18s;"></div><div class="sf sf1" style="top:-2.26em; left:21.88vw; animation-delay:-0.71s;"></div><div class="sf sf3" style="top:-4.83em; left:31.88vw; animation-delay:-16.70s;"></div><div class="sf sf3" style="top:-4.04em; left:32.17vw; animation-delay:-5.17s;"></div><div class="sf sf0" style="top:-1.66em; left:19.85vw; animation-delay:-1.40s;"></div><div class="sf sf0" style="top:-1.68em; left:54.84vw; animation-delay:-8.82s;"></div><div class="sf sf3" style="top:-4.66em; left:25.86vw; animation-delay:-10.96s;"></div><div class="sf sf3" style="top:-4.09em; left:59.00vw; animation-delay:-12.25s;"></div><div class="sf sf0" style="top:-1.75em; left:39.72vw; animation-delay:-4.51s;"></div><div class="sf sf0" style="top:-1.52em; left:70.18vw; animation-delay:-15.09s;"></div><div class="sf sf2" style="top:-3.80em; left:1.60vw; animation-delay:-3.01s;"></div><div class="sf sf3" style="top:-4.91em; left:50.50vw; animation-delay:-19.28s;"></div><div class="sf sf1" style="top:-2.66em; left:10.43vw; animation-delay:-2.12s;"></div><div class="sf sf2" style="top:-3.42em; left:12.98vw; animation-delay:-5.57s;"></div><div class="sf sf1" style="top:-2.36em; left:93.25vw; animation-delay:-16.49s;"></div><div class="sf sf2" style="top:-3.68em; left:49.98vw; animation-delay:-10.09s;"></div><div class="sf sf1" style="top:-2.61em; left:37.91vw; animation-delay:-13.74s;"></div><div class="sf sf1" style="top:-2.76em; left:72.46vw; animation-delay:-2.85s;"></div><div class="sf sf2" style="top:-3.49em; left:46.55vw; animation-delay:-18.56s;"></div><div class="sf sf0" style="top:-1.48em; left:43.70vw; animation-delay:-4.87s;"></div><div class="sf sf2" style="top:-3.61em; left:54.52vw; animation-delay:-0.10s;"></div><div class="sf sf3" style="top:-4.53em; left:22.79vw; animation-delay:-16.65s;"></div><div class="sf sf0" style="top:-1.22em; left:24.51vw; animation-delay:-19.97s;"></div><div class="sf sf1" style="top:-2.32em; left:64.69vw; animation-delay:-6.96s;"></div><div class="sf sf2" style="top:-3.85em; left:72.26vw; animation-delay:-11.81s;"></div><div class="sf sf1" style="top:-2.57em; left:10.11vw; animation-delay:-8.13s;"></div><div class="sf sf1" style="top:-2.61em; left:69.90vw; animation-delay:-8.80s;"></div><div class="sf sf2" style="top:-3.04em; left:54.50vw; animation-delay:-1.51s;"></div>
</main>

<!-- ga -->
<script>
(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','//www.google-analytics.com/analytics.js','ga');
ga('create', 'UA-69522494-1', 'auto');
ga('set', 'anonymizeIp', true);
ga('send', 'pageview');
</script>
<!-- /ga -->

</body>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>16931171414113</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
